<a href="https://colab.research.google.com/github/dnguyend/MiscCollection/blob/main/colab/EinsteinRicciStiefel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ricci Tensor and Einstein metrics on Stiefel manifold $St_{p,n}$ with left-invariant diagonal metrics
We show how to use a simple formula for the Ricci tensor, when the metric on $St{p, n}$ is given by $\langle (A, B), (A, B)\rangle_P = \sum_{i<j}a_{ij}^2t_{ij} +\sum_{i=1}^{n-p}\sum_{j=1}^pb_{ij}^2s_j$, where a tangent vector to the Stiefel manifold is represented by two matrices $A$ adn $B$, with $A$ anti-symmetric in $R^{p\times p}$ and $B\in R^{(n-p)\times p}$.

The Ricci tensor is given by $Ric(A, A) = \sum_{i<j}a_{ij}^2c_{ij} + \sum b_{ij}^2f_j$ with
$$c_{ij} = n-2+ \frac{n-p}{2}\frac{t_{ij}^2 -s_i^2-s_j^2}{s_is_j} + \frac{1}{2}\sum_{l\neq i, l\neq j}\frac{t_{ij}^2-t_{il}^2-t_{jl}^2}{t_{il}t_{jl}}
$$
$$f_j = n-2 + \frac{1}{2}\sum\frac{s_j^2 - t_{jl}^2 - s_l^2}{s_l t_{jl}}$$
to solve for Einstein metric on $St_{p, n}$

We can run the codes cell by cell but we can just follow the code without the run. Some cell does not return values as it may contain definition only.

First - we code implementing the formula. We keep the metric as either a vector $TS$ of size $p(p-1)/2+p$, or a pair of $p\times p$ symmetric matrix with diagonal zero and a diagonal metric size $p$.

In [1]:
import numpy as np
import contextlib
from scipy.optimize import minimize


def makeTmat(Tdiag):
    p = .5 * (1 + np.sqrt(1 + 8 * len(Tdiag)))
    p = int(np.round(p))

    Tmat = np.zeros((p, p))
    tx = np.triu_indices(p, 1)
    Tmat[tx] = Tdiag
    Tmat[tx[1], tx[0]] = Tdiag
    return Tmat


def makeTSmat(v, p):
    nupa = p*(p-1)//2    
    
    uu = np.zeros((p+1, p+1))
    uu[:p, :p] = makeTmat(v[:nupa])
    uu[:-1, -1] = v[nupa:]
    uu[-1, :-1] = v[nupa:]
    return uu

def RicciCoeff(TS, n, p):
    nupa = p*(p-1)//2
    Tdiag = TS[:nupa]
    uu = makeTmat(Tdiag)
    uui = makeTmat(1/Tdiag)
    Sdiag = TS[nupa:]

    tx = np.triu_indices(p , 1)

    kfA = np.zeros(nupa)
    for ss in range(nupa):
        i, j = tx[0][ss], tx[1][ss]
        kfA[ss] += n - 2
        kfA[ss] += -0.5*(n-p)*(Sdiag[i]/Sdiag[j] + Sdiag[j]/Sdiag[i])
        kfA[ss] += 0.5*(n-p)*uu[i, j]*uu[i, j]/Sdiag[i]/Sdiag[j]

        for aa in range(p):
            if aa != j and aa != i:
                kfA[ss] += -0.5*(uu[i, aa]/uu[j, aa] + uu[j, aa]/uu[i, aa])
                kfA[ss] += +0.5*uu[i, j]*uu[i, j]/uu[j, aa]/uu[i, aa]

    kfB = np.zeros((n-p, p))
    for i in range(n-p):
        for j in range(p):
            kfB[i, j] += n - 2                    
            kfB[i, j] += -0.5*((1/Sdiag)*uu)[j, :].sum()
            kfB[i, j] += 0.5*Sdiag[j]*Sdiag[j]*(1/Sdiag*uui)[j, :].sum()
            kfB[i, j] += -0.5*(Sdiag*uui)[j, :].sum()                

    return np.concatenate([kfA, kfB.reshape(-1)])




Now we implement the cost function,  $F = \sum_{u}(\lambda_{u} -\frac{\sum_{u}\lambda_{u}}{\frac{1}{2}p(p+1)})^2$ where $\lambda_u$ is either 
$\lambda_{ij} = r_{ij}/t_{ij}$ for $i<j$ or $f_j/s_j$ for $1\leq j\leq p$, that means $u$ runs over the dimensions of the parameter space.
 We work with the variable $v_{ij} = 1/t_{ij}$ or $1/s_j$, slightly easier to implement. We set the last coefficient of $v$ to $1$ so the variable is actually a vector of size $p(p+1)/2-1$, called $z$. fgrad returns both the value and the gradient of the cost function in the variable $z$

In [2]:
def fgrad_main(ITS, n, p):
    """Cost function is
    ||(RicciCoeffInv(ITS)*ITS)[1:] - (RicciCoeffInv(ITS)*ITS)[:-1]||^2
    """
    nupa = p*(p-1)//2
    
    def idx(ll, i):
        """ need ll != i
        """
        if ll < i:
            return ll*(2*p-ll-1)//2 + i-ll-1
        elif ll > i:
            return i*(2*p-i-1)//2 + ll-i-1
        else:
            raise(IndexError('not found %d %d' % (ll, i)))

    mdim = nupa + p
    ITdiag = ITS[:nupa]
    # uu = makeUmat0(1/ITdiag)
    # uui = makeUmat0(ITdiag)
    ISdiag = ITS[nupa:]

    tx = np.triu_indices(p , 1)

    jac = np.zeros((mdim, mdim))
    kfA = np.zeros(nupa)
    kfB = np.zeros(p)

    for ss in range(nupa):
        i, j = tx[0][ss], tx[1][ss]
        c = (n - 2)
        c += -0.5*(n-p)*(ISdiag[i]/ISdiag[j] + ISdiag[j]/ISdiag[i])
        c += 0.5*(n-p)/ITdiag[ss]/ITdiag[ss]*ISdiag[i]*ISdiag[j]

        jac[nupa+i, ss] += -0.5*(n-p) *\
            (1/ISdiag[j] - ISdiag[j]/ISdiag[i]/ISdiag[i])
        jac[nupa+j, ss] += -0.5*(n-p) *\
            (1/ISdiag[i] - ISdiag[i]/ISdiag[j]/ISdiag[j])

        jac[ss, ss] += -2*0.5*(n-p)/ITdiag[ss]/ITdiag[ss]/ITdiag[ss] *\
            ISdiag[i]*ISdiag[j]            
        jac[nupa+i, ss] += 0.5*(n-p)/ITdiag[ss]/ITdiag[ss]*ISdiag[j]
        jac[nupa+j, ss] += 0.5*(n-p)/ITdiag[ss]/ITdiag[ss]*ISdiag[i]

        for aa in range(p):
            if aa != j and aa != i:
                i_ia = idx(i, aa)
                i_ja = idx(j, aa)

                c += -0.5*(ITdiag[i_ia]/ITdiag[i_ja] + ITdiag[i_ja]/ITdiag[i_ia])
                c += +0.5*ITdiag[i_ja]*ITdiag[i_ia]/ITdiag[ss]/ITdiag[ss]

                jac[i_ia, ss] += -0.5*(
                    1/ITdiag[i_ja] - ITdiag[i_ja]/ITdiag[i_ia]/1/ITdiag[i_ia])
                jac[i_ja, ss] += -0.5*(
                    1/ITdiag[i_ia] - ITdiag[i_ia]/ITdiag[i_ja]/1/ITdiag[i_ja])

                jac[ss, ss] += -2*0.5*ITdiag[i_ja]*ITdiag[i_ia] /\
                    ITdiag[ss]/ITdiag[ss]/ITdiag[ss]
                jac[i_ia, ss] += 0.5*ITdiag[i_ja]/ITdiag[ss]/ITdiag[ss]
                jac[i_ja, ss] += 0.5*ITdiag[i_ia]/ITdiag[ss]/ITdiag[ss]
        kfA[ss] += c

    for j in range(p):            
        cB = n - 2
        for aa in range(p):
            if aa == j:
                continue
            i_ja = idx(j, aa)
            cB += -0.5*ISdiag[aa]/ITdiag[i_ja]
            cB += 0.5*ISdiag[aa]*ITdiag[i_ja]/ISdiag[j]/ISdiag[j]
            cB += -0.5/ISdiag[aa]*ITdiag[i_ja]

            jac[nupa + aa, nupa+j] += -0.5/ITdiag[i_ja]
            jac[i_ja, nupa+j] += 0.5*ISdiag[aa]/ITdiag[i_ja]/ITdiag[i_ja]

            jac[nupa + aa, nupa+j] += 0.5*ITdiag[i_ja]/ISdiag[j]/ISdiag[j]
            jac[nupa + j, nupa+j] += -2*0.5*ISdiag[aa]*ITdiag[i_ja] /\
                ISdiag[j]/ISdiag[j]/ISdiag[j]
            jac[i_ja, nupa+j] += 0.5*ISdiag[aa]/ISdiag[j]/ISdiag[j]

            jac[nupa + aa, nupa+j] += 0.5*ITdiag[i_ja]/ISdiag[aa]/ISdiag[aa]
            jac[i_ja, nupa+j] += -0.5/ISdiag[aa]
        kfB[j] += cB

    kf = np.concatenate([kfA, kfB])
    # return kf, jac

    lbd = kf*ITS
    cost1 = (lbd*lbd).sum()
    grad1 = 2*jac@(kf*ITS*ITS) + 2*kf*kf*ITS
    # cost1 = lbd.sum()
    # grad1 = jac@ITS + kf

    cost2 = - mdim*lbd.mean()*lbd.mean()
    grad2 = -2*lbd.mean()*(jac@ITS + kf)
    return cost1 + cost2, (grad1 + grad2).T




A quick check of the gradient:


In [3]:
p = 5
n = 7
nup = p*(p-1)//2
dlt = 1e-6
xi = np.random.randn(nup+p)
x0 = np.random.randn(nup+p)
x0 = 1 + x0 - x0.min()

ff, gg = fgrad_main(x0, n, p)
ff1, gg1 = fgrad_main(x0+dlt*xi, n, p)

print('check gradient %.5f %.5f' % 
      ((ff1 - ff)/dlt, np.sum(gg*xi)))


check gradient -1730.53156 -1730.53551


Now we run the optimization. We demonstrate that the code recover some known Einstein metrics from the literature. There may be a lot of repeated results, something we hope to avoid in the future. We try to avoid repetition by dividing the metric by it smallest coefficient then compare to see if a new run produce a new metric. We start with 100 runs

In [4]:
@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally:
        np.set_printoptions(**original)


def pprint(mat, precision=4):
    """pretty print numpy matrix
    """
    with printoptions(precision=precision, suppress=True):
        print(mat)

def do_one_full(n, p, N=100):
    rets = {}
    nupa = (n-p)*p
    mdim = (n-p)*p + p
    bounds = (mdim-1)*[(1e-6, np.inf)]

    def fg_x1(x):
        """ search over the full space except for one coefficient set to 1
        """
        xs = np.concatenate([x, np.ones(1)])
        c, grd = fgrad_main(xs, n, p)
        return c, grd[:-1]

    cnt = 0
    while cnt < N:
        x0 = np.random.randint(1, 100, mdim-1)
        x0 = 1 + x0 - x0.min()
        ret = minimize(fg_x1, x0=x0, jac=True, tol=1e-15, bounds=bounds)
        if ret['success']:
            topt = np.concatenate([1/ret['x'], np.ones(1)])
            topt = topt / topt[-p:].min()
            rets[cnt] = {
                'cost': fg_x1(ret['x'])[0],
                'x': ret['x'],
                'metric': topt,
                'rtio': RicciCoeff(topt, n, p)[:mdim] / topt
            }
            cnt += 1
        
    sst = np.zeros((N, 5))
    cnt = 0
    for i in range(len(rets)):
        if (rets[i]['cost'] < 1e-3) and (rets[i]['rtio'].min() > 0):
            sst[cnt, :] = [i, rets[i]['cost'],
                           rets[i]['metric'].min(),
                           rets[i]['metric'].mean(),
                           rets[i]['metric'].max() -
                           rets[i]['metric'][-p:].mean()]
            cnt += 1
    sst = sst[:cnt, :]
    sst = sst[np.argsort(sst[:, 3])]
    last = - 1
    good = []
    for i in range(sst.shape[0]):
        ix = int(sst[i, 0])
        # print(i, sst[i, 3], last, np.abs(sst[i, 3] - last), rets[ix]['metric'])
        if True and np.abs(sst[i, 3] - last) > 1e-3:
            last = sst[i, 3]
            good.append(ix)
            
    with open('stiefl_x1.%s.npz' % 
              '_'.join([str(n), str(p)]), 'wb') as ff:
        np.savez_compressed(ff, sst=sst, rets=rets)
            
    def summarize(ixx):
        ssT = np.sort(rets[good[ixx]]['metric'][:-p])
        ssS = np.sort(rets[good[ixx]]['metric'][-p:])
        diffT = np.abs(ssT[1:]/ssT[:-1]-1)
        print('Doing %d' % ixx)
        # print(ss)
        markT = np.where(diffT > 1e-3)[0]
        last = 0
        tc = 0
        print('T values')
        for i in range(markT.shape[0]):
            if markT[i] == last:
                val = np.mean(ssT[last:last + 1])
                cnt = 1
                print(val, cnt)
                last = last + 1
                tc += 1
            else:
                val = np.mean(ssT[last:markT[i]])
                cnt = markT[i] - last+1
                print(val, cnt)
                last = markT[i]+1
                tc += 1
        if last < nupa:
            val = np.mean(ssT[last:])
            tc += 1
            print(val, ssT[last:].shape[0])

        diffS = np.abs(ssS[1:]/ssS[:-1]-1)
        markS = np.where(diffS > 1e-3)[0]
        last = 0
        sc = 0
        print('S values')        
        for i in range(markS.shape[0]):
            if markS[i] == last:
                val = np.mean(ssS[last:last + 1])
                cnt = 1
                print(val, cnt)
                last = last + 1
                sc += 1
            else:
                val = np.mean(ssS[last:markS[i]])
                cnt = markS[i] - last+1
                print(val, cnt)
                last = markS[i]+1
                sc += 1
        if last < p:
            val = np.mean(ssS[last:])
            sc += 1
            print(val, ssS[last:].shape[0])
        print('totCnt=%d cntT=%d cntS=%d' % (tc + sc, tc, sc))
        rts = rets[good[ixx]]['rtio']
        print(ixx, good[ixx], rts.mean(), np.std(rts))
        pprint(makeTSmat(rets[good[ixx]]['metric'], p))

    for ixx in range(len(good)):
        summarize(ixx)
    return rets, sst

In [5]:
n = 7
p = 5

np.random.seed(1)    
N = 30
rets = do_one_full(n, p, N)


Doing 0
T values
0.14940744874791909 3
0.6202009958204809 6
0.8317709034580707 1
S values
1.0000000002801077 3
1.1393375781804318 2
totCnt=5 cntT=3 cntS=2
0 0 3.884385831214892 9.473978206902186e-08
[[0.     0.6202 0.6202 0.8318 0.6202 1.1393]
 [0.6202 0.     0.1494 0.6202 0.1494 1.    ]
 [0.6202 0.1494 0.     0.6202 0.1494 1.    ]
 [0.8318 0.6202 0.6202 0.     0.6202 1.1393]
 [0.6202 0.1494 0.1494 0.6202 0.     1.    ]
 [1.1393 1.     1.     1.1393 1.     0.    ]]
Doing 1
T values
1.274292283522041 10
S values
1.000000315627789 5
totCnt=2 cntT=1 cntS=1
1 29 2.451415458285767 2.9359493516282874e-08
[[0.     1.2743 1.2743 1.2743 1.2743 1.    ]
 [1.2743 0.     1.2743 1.2743 1.2743 1.    ]
 [1.2743 1.2743 0.     1.2743 1.2743 1.    ]
 [1.2743 1.2743 1.2743 0.     1.2743 1.    ]
 [1.2743 1.2743 1.2743 1.2743 0.     1.    ]
 [1.     1.     1.     1.     1.     0.    ]]
Doing 2
T values
0.34634602458685737 3
1.301362975725612 1
2.9481855721977013 6
S values
1.0 2
2.856126521279078 3
totCnt=5

There are 3 metrics found - some matches with Jensen and Arvanitoyeorgos et. al. and there is a new metric.

In [6]:
def verify_stf_75_1332():    
    TS = np.array([3.86531131, 4.15568173, 3.86531131, 3.8653113 , 1.78072524,
       0.4995625 , 0.49956253, 1.78072524, 1.78072524, 0.4995625 ,
       1.        , 3.79800143, 4.09222011, 3.79800143, 3.79800141])
    
    n = 7
    p = 5
    print(TS.shape, n, p)
    Tmat = makeTSmat(TS, p)
    rcf = RicciCoeff(TS, n, p)[:p*(p+1)//2]
    print("coefficients")
    pprint(Tmat)
    lbd = rcf/TS
    print("Ratios")
    pprint(makeTmat(lbd))
    
    print('Ratio =%.5f standard deviation = %f' % (lbd.min(), np.std(lbd)))
verify_stf_75_1332()

(15,) 7 5
coefficients
[[0.     3.8653 4.1557 3.8653 3.8653 1.    ]
 [3.8653 0.     1.7807 0.4996 0.4996 3.798 ]
 [4.1557 1.7807 0.     1.7807 1.7807 4.0922]
 [3.8653 0.4996 1.7807 0.     0.4996 3.798 ]
 [3.8653 0.4996 1.7807 0.4996 0.     3.798 ]
 [1.     3.798  4.0922 3.798  3.798  0.    ]]
Ratios
[[0.    1.131 1.131 1.131 1.131 1.131]
 [1.131 0.    1.131 1.131 1.131 1.131]
 [1.131 1.131 0.    1.131 1.131 1.131]
 [1.131 1.131 1.131 0.    1.131 1.131]
 [1.131 1.131 1.131 1.131 0.    1.131]
 [1.131 1.131 1.131 1.131 1.131 0.   ]]
Ratio =1.13100 standard deviation = 0.000000


To reduce the number of variables (instead of $p(p+1)/2$) we divide the $p\times p$ to blocks with coefficients in the same blocks only. This is a simple application of the chain rule. We run over all partitions for $p=5$. Again there are a lot of repetition but we hope this demonstrates the method is sufficiently robust.

## The very large values on some output is because the minimizer find solutions at the boundary of the search domain. With appropriate normalization, these corresponds to degenerate metrics where some coefficients $t_{ij} = 0$, so they should be ignored.

## Solution from a subpartition may include solutions of its parent partitions.

## the next cell run for all flags of $St{5}{7}$, 100 iteration each. The result is normalized dividing by the smallest $s$ value. The cell after normalizes the known examples (Jensen and Arv. et. al) and compare to see if we find those known examples, or if we find new example

In [7]:
def partitions(n, II=1):
  # list all partition of N
    yield (n,)
    for i in range(II, n//2 + 1):
        for p in partitions(n-i, i):
            yield (i,) + p  

def makeFF_idx(flgs):
    def fill_up(val, d):
        """ return a matrix with upper triangle block filled with val
        """
        ret = np.zeros((d, d))
        ret[np.triu_indices(d, 1)] = val
        return ret

    q = flgs.shape[0]-1
    p = flgs[:-1].sum()
    mat = np.zeros((p, p))  
    rstart = 0
    cstart = 0
    wdim = 0
    # bvec = np.ones(p)
    
    for i in range(q):
        cstart = rstart
        for j in range(i, q):
            if i == j:        
                mat[rstart:rstart + flgs[i], cstart:cstart + flgs[i]] =\
                    fill_up(wdim, flgs[i])
                if flgs[i] > 1:
                    wdim += 1
            else:
                mat[rstart:rstart + flgs[i], cstart:cstart + flgs[j]] = wdim
                wdim += 1
            cstart += flgs[j]
        # bvec[i] = wdim
        # wdim += 1
        rstart += flgs[i]
    bvec = np.concatenate([np.array(flgs[i]*[wdim+i]) for i in range(q)])
    return np.concatenate(
        [mat[np.triu_indices(p, 1)][:], bvec]),\
        (q+2)*(q+1)//2 - 2 - np.where(flgs == 1)[0].shape[0]
    
    
def makeUnique(alist):
    return [float(a) for a in np.unique(['%.4f' % a for a in alist])]


def do_one_flag(flgs, N, w0=None):
    """ flag adds up to n, total is q+1 blocks, with the last block form
    the group K
    the first q are blocks of the A matrix
    """
    print('Doing %s' % str(flgs))
    n = flgs.sum()
    q = flgs.shape[0] - 1
    p = flgs[:-1].sum()
    nupa = p*(p-1)//2
    mdim = nupa + p
    vec_idx, wdim = makeFF_idx(flgs)

    def x2u(x):
        u = np.ones(mdim)
        u[:-1] = x
        return u

    def w2zFF(w):
        # careful with this one
        z = np.ones(mdim)
        for i in range(w.shape[0]):
            z[np.where(vec_idx == i)] = w[i]
        return z

    def fgrwFF(w):
        ff, gg = fgrad_main(w2zFF(w), n, p)
        return ff, np.array(
            [gg[np.where(vec_idx == i)].sum() for i in range(w.shape[0])])

    bounds = wdim*[(1e-6, np.inf)]

    rets = {}
    cnt = 0
    while cnt < N:
        if w0 is None:
            xw0 = np.random.randn(wdim)
            # xw0 = np.random.randint(1, 100, wdim)
            # xw0 = 1 + x0 - x0.min()            
            xw0 = 1 + xw0 - xw0.min()
            # print(xw0)
        else:
            xw0 = w0.copy()
        ret = minimize(fgrwFF, x0=xw0, jac=True, tol=1e-10, bounds=bounds)
        if ret['success']:
            topt = w2zFF(1/ret['x'])
            topt = topt / topt[-p:].min()
            
            rets[cnt] = {
                'cost': fgrwFF(ret['x'])[0],
                'x': ret['x'],
                'metric': topt,
                'rtio': RicciCoeff(topt, n, p)[:mdim] / topt
                }
            cnt += 1

    sst = np.zeros((N, 5))
    cnt = 0
    for i in range(len(rets)):
        if (rets[i]['cost'] < 1e-3) and (rets[i]['rtio'].min() > 0):
            sst[cnt, :] = [i, rets[i]['cost'],
                           rets[i]['metric'].min(),
                           rets[i]['metric'].mean(),
                           rets[i]['metric'].max() -
                           rets[i]['metric'][-p:].mean()]            
            cnt += 1
            
    with open('stiefl.%s.%s.npz' % (
            '_'.join([str(n), str(p)]),
            '_'.join(
                [str(i) for i in flgs])), 'wb') as ff:
        np.savez_compressed(ff, sst=sst, rets=rets)

    return rets, sst
    
def test_all_flags():
    n = 7
    p = 5
    np.random.seed(1)
    N = 100
    all_flags = sorted([a for a in partitions(p)],
                       key=lambda x: len(x))[1:-1]
    n_p = n - p
    for flgs0 in all_flags:
        flgs = np.array(list(flgs0) + [n_p])
        f = do_one_flag(flgs, N)


test_all_flags()

Doing [1 4 2]
Doing [2 3 2]
Doing [1 1 3 2]
Doing [1 2 2 2]
Doing [1 1 1 2 2]


## The above save data to the current directory, '/content' we load it and compare with known metrics.

In [9]:
load_dir = '/content/'
def analyze_all_flags():
    n = 7
    p = 5
    import os
    all_files = [a for a in os.listdir(load_dir) \
                 if a.startswith(
                         'stiefl.%s' % '_'.join([str(n), str(p)]))]
    
    good_list = {}
    cnt = 0
    # for f in ['stiefl.7_5.1_1_3_2.npz']:
    for f in all_files:    
        af = np.load('%s%s' % (load_dir, f), allow_pickle=True)
        ll = dict(af['rets'].tolist())
        for i in ll:
            if np.abs(ll[i]['cost']) < 1e-3 and np.std(ll[i]['rtio']) < 1e-2 \
               and np.min(ll[i]['rtio']) > 1e-4:
                good_list[cnt] = [f, i, ll[i]['cost'], ll[i]['metric'], ll[i]['rtio']]
                cnt += 1
    good_list_k = sorted(good_list, key=lambda x: good_list[x][4].mean())
    
    nupa = p*(p-1)//2
    js = [(n-2 - np.sqrt((n-2)*(n-2)-(n-1)*(p-2)))/(n-1),
          (n-2 + np.sqrt((n-2)*(n-2)-(n-1)*(p-2)))/(n-1)]
    
    vecs = {}
    for i in range(len(js)):
        vecs['j%s' % i] = np.array(nupa*[js[i]] + p*[1])
        
    s232 = [[1, 1.13934, .620201, .831771, .149407],
            [1, .350124, 1.03223, .455639, .121264]]
    
    for i in range(len(s232)):
        vecs['s232%s' % i] = np.array(
            6*[s232[i][2]] + [s232[i][3]] + 3*[s232[i][4]]
            + 2*[s232[i][1]] + 3*[1])
    
    s42 = [[1, .253386, 1.01652, .245146],
           [1, 1.16137, .66907, .291175]]

    for i in range(len(s42)):
        vecs['s42%s' % i] = np.array(
            4*[s42[i][2]] + 6*[s42[i][3]] + [s42[i][1]] + 4*[1])

    def sort_by_sec(vv):
        return np.concatenate(
            [np.sort(vv[:-p]), np.sort(vv[-p:])])
                                      
    for k in vecs:
        vecs[k] = vecs[k] / vecs[k][-p:].min()
        vecs[k] = sort_by_sec(vecs[k])            

    # check old new
    good_list_new = []
    good_list_found = []
    for i in range(len(good_list_k)):
        found = False
        for kk in vecs:
            diff = np.linalg.norm(
                sort_by_sec(good_list[good_list_k[i]][3]) -
                vecs[kk])
            if diff < 5e-3:
                found = True
                if kk not in good_list_found:
                    print('Found %s' % kk)            
                    print(good_list[good_list_k[i]][3])
                    good_list_found.append(kk)
        if not found:
            good_list_new.append(good_list_k[i])
    print('new list')
    for kk in good_list_new:
        print(good_list[kk][3])
        
    def summarize_one(ixx, i):
        ssT = np.sort(good_list[ixx][3][:-p])
        ssS = np.sort(good_list[ixx][3][-p:])
        diffT = np.abs(ssT[1:]/ssT[:-1]-1)
        print('Summarize %d %d %s' % (i, ixx, good_list[ixx][0]))
        # print(ss)
        markT = np.where(diffT > 1e-3)[0]
        last = 0
        tc = 0
        print(ixx, good_list[ixx])
        
        print('T values')
        for i in range(markT.shape[0]):
            if markT[i] == last:
                val = np.mean(ssT[last:last + 1])
                cnt = 1
                print(val, cnt)
                last = last + 1
                tc += 1
            else:
                val = np.mean(ssT[last:markT[i]])
                cnt = markT[i] - last+1
                print(val, cnt)
                last = markT[i]+1
                tc += 1
        if last < nupa:
            val = np.mean(ssT[last:])
            tc += 1
            print(val, ssT[last:].shape[0])

        diffS = np.abs(ssS[1:]/ssS[:-1]-1)
        markS = np.where(diffS > 1e-3)[0]
        last = 0
        sc = 0
        print('S values')        
        for i in range(markS.shape[0]):
            if markS[i] == last:
                val = np.mean(ssS[last:last + 1])
                cnt = 1
                print(val, cnt)
                last = last + 1
                sc += 1
            else:
                val = np.mean(ssS[last:markS[i]])
                cnt = markS[i] - last+1
                print(val, cnt)
                last = markS[i]+1
                sc += 1
        if last < p:
            val = np.mean(ssS[last:])
            sc += 1
            print(val, ssS[last:].shape[0])
        print('totCnt=%d cntT=%d cntS=%d' % (tc + sc, tc, sc))
        if tc > 3:
            print(makeTSmat(good_list[ixx][3], p))

    cnt = 0
    last = np.zeros(good_list[good_list_k[0]][3].shape[0])

    for i in range(len(good_list_new)):
        current = sort_by_sec(good_list[good_list_k[i]][3])
        diff = np.linalg.norm(current - last)
        if diff > 5e-3:
            idxx = good_list_new[i]
            summarize_one(idxx, i)
            last = current
        
analyze_all_flags()


Found s420
[4.011778   0.96748679 0.96748356 0.96748356 4.01177902 4.01177824
 4.01177824 0.96748734 0.96748734 0.96748359 3.94656201 1.
 3.94656313 3.94656199 3.94656199]
Found s2321
[0.34634691 0.34634874 2.94820027 2.94820027 0.3463504  2.9481986
 2.9481986  2.94820005 2.94820005 1.30136401 2.85614075 2.85614207
 2.85614357 1.         1.        ]
Found j1
[1.27438591 1.27438591 1.27437194 1.27437194 1.27450574 1.27449122
 1.27449122 1.27449122 1.27449122 1.27447621 1.         1.00016101
 1.00016101 1.00014159 1.00014159]
Found s2320
[0.62020197 0.62020224 0.14940751 0.14940751 0.83177252 0.62020201
 0.62020201 0.62020227 0.62020227 0.14940776 1.         1.13933705
 1.13933831 1.0000001  1.0000001 ]
Found s421
[0.66907277 0.29117501 0.29117521 0.29117521 0.66907273 0.66907269
 0.66907269 0.29117511 0.29117511 0.29117519 1.00000019 1.16136851
 1.         1.00000011 1.00000011]
Found j0
[0.39237512 0.39237512 0.39237544 0.39237544 0.39237505 0.39237534
 0.39237534 0.39237534 0.39237534

So there is a new example listed last - corresponding to the flag $(1, 1, 3, 2)$, the same example found above by scanning over all variables, in different order.